In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #for no tensorflow warnings"

2021-11-18 20:52:28.618933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 20:52:28.618960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Make and train model on MNIST

In [ ]:
# load dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 28, 28).astype("float32") / 255
x_test = x_test.reshape(10000, 28, 28).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28,1)))
model.add(tf.keras.layers.Dense(50, activation="relu"))
model.add(tf.keras.layers.Dense(50, activation="relu"))
model.add(tf.keras.layers.Dense(50, activation="relu"))
model.add(tf.keras.layers.Dense(10))
model.summary()

In [ ]:
def softmax_loss():        
    def loss(y_true, y_pred):
        y_soft = tf.keras.activations.softmax(y_pred, axis=1)
        loss_soft=keras.losses.SparseCategoricalCrossentropy()(y_true, y_soft)
        return loss_soft
    return loss

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=softmax_loss(), 
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=20,
    validation_data=(x_val, y_val),
    verbose=2,    
)

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1)

# save and load model

In [ ]:
model.save('MNIST_2x50.h5')

In [ ]:
new_model = tf.keras.models.load_model('MNIST_2x50.h5', custom_objects={'loss': softmax_loss()})

# use model in DeepAbstract

In [ ]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #for no tensorflow warnings"
sys.path.append(os.path.abspath('../..'))

from src import *
from models import *
from src.main import Cluster_Class, set_file_model, set_keras_model
from src.models import Keras_Model

In [ ]:
m = set_keras_model(new_model) # layer printed last does not get clustered

In [ ]:
m.test_accuracy(True)

In [ ]:
cc = Cluster_Class(m, [50-3, 50-2, 50-1], cl_method="dbscan")
cc.perform_clustering()